In [1]:
from metric import evaluate
from metric_preprocess import load_images_from_folder, save_predictions_to_csv, yolo8_to_gt_csv
from solution import predict
import time

/home/bowie/miniconda3/envs/ml/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bowie/miniconda3/envs/ml/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
image_folder = "/mnt/datadisk/Dataset/dataset/03_part_dataset_human_rescue/private/images"
labels_folder = "/mnt/datadisk/Dataset/dataset/03_part_dataset_human_rescue/private/labels"

predict_csv_file = "/home/bowie/repos/ml/inference/eval/predict.csv"

gt_csv_file = "/home/bowie/repos/ml/inference/eval/gt.csv"


In [ ]:
import os, time, gc
from pathlib import Path
from typing import List
import numpy as np
from PIL import Image
import torch

# image_folder и predict_csv_file должны быть заданы заранее

predictions: List[List[dict]] = []
image_paths: List[str] = []
processing_times: List[float] = []

for i, p in enumerate(sorted(Path(image_folder).iterdir()), 1):
    if p.suffix.lower() not in {'.jpg', '.jpeg', '.png'}:
        continue

    img_path = str(p)
    with Image.open(img_path) as img:
        img = img.convert('RGB')
        arr = np.array(img, copy=False)  # HWC RGB, np.uint8

    start = time.time()
    with torch.no_grad():
        preds = predict(arr)  # твоя функция: вернёт List[List[dict]]
    duration = time.time() - start

    predictions.append(preds[0])       # только для одного изображения
    image_paths.append(img_path)
    processing_times.append(duration)

    if i % 64 == 0:
        gc.collect()

# Сохраняем всё разом
save_predictions_to_csv(predictions, image_paths, processing_times, predict_csv_file)


detections after segmentation 15
tensor([0.8183, 0.0777, 0.9113, 0.0605, 0.0181, 0.1508, 0.7138, 0.7584, 0.0125, 0.0391, 0.8684, 0.7279, 0.1949, 0.1087, 0.1491])
Running yolo at 12 detections
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
detections after filtering 4
detections after thresholding boxes 4
detections after removing copies 4
detections after segmentation 18
tensor([0.4147, 0.8106, 0.2724, 0.0899, 0.8956, 0.6480, 0.8030, 0.0384, 0.1753, 0.0315, 0.4039, 0.0145, 0.5843, 0.0375, 0.6658, 0.5731, 0.5842, 0.5259])
Running yolo at 15 detections
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
detections after filtering 5
detections after thresholding boxes 4
detections after removing copies 4
detections after segmentation 21
tensor([0.7959, 0.7551, 0.8645, 0.2299, 0.3214, 0.2414, 0.7026, 0.8480, 0.1124, 0.8953, 0.3581, 0.1748, 0.0086, 0.0198, 0.7491, 0.5838, 0.6830, 0.0298, 0.8486, 0.9524, 0.3043])
Running yolo at 16 detections
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0]
detections after

In [ ]:
"""
    Creating ground truth file
"""
yolo8_to_gt_csv(labels_folder, image_folder, output_csv=gt_csv_file)

[✓] GT-файл сохранён: /home/bowie/repos/ml/eval/gt.csv


In [ ]:
evaluate(predict_csv_file, gt_csv_file)

(np.float64(0.0020923077),
 {0.3: 0.208,
  0.37: 0.208,
  0.44: 0.208,
  0.51: 0.208,
  0.58: 0.208,
  0.65: 0.208,
  0.72: 0.208,
  0.79: 0.208,
  0.86: 0.104,
  0.93: 0.0},
 {0.3: 99.792,
  0.37: 99.792,
  0.44: 99.792,
  0.51: 99.792,
  0.58: 99.792,
  0.65: 99.792,
  0.72: 99.792,
  0.79: 99.792,
  0.86: 99.896,
  0.93: 100.0},
 2.93)